In [ ]:
# standard imports
import numpy as np
import math
import scipy
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh


# utility functions
from isentropic_navier_stokes import Q,L,inner_product
from utilities_modified import *

# Setting Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,[0.5,0])

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_comp_air/mach 0.01/"
data_dict = read_csv_data(data_dir,delay=5000,subsampling=2,max_size=180)

# Data Preparation

In [ ]:
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]
u = np.divide(data_dict['Momentum_x'],data_dict['Density'])
v = np.divide(data_dict['Momentum_y'],data_dict['Density'])
a = data_dict['Mach']
n = mesh.n
N = mesh.N

In [ ]:
q = np.vstack([u,v,a])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

# Plotting sample Stencil

In [ ]:
sample = 75
fig,ax = plt.subplots(1,1,figsize=(6,6))

u = mesh.nodes[sample].u
b = mesh.nodes[sample].b
r = mesh.nodes[sample].r
l = mesh.nodes[sample].l

if u!=None:
    u_r = mesh.nodes[u].r
    u_l = mesh.nodes[u].l
    plt.scatter(mesh.nodes[u].x,mesh.nodes[u].y,c='g')
    plt.scatter(mesh.nodes[u_r].x,mesh.nodes[u_r].y,c='black')
    plt.scatter(mesh.nodes[u_l].x,mesh.nodes[u_l].y,c='black')

if b!=None:
    b_r = mesh.nodes[b].r
    b_l = mesh.nodes[b].l
    plt.scatter(mesh.nodes[b].x,mesh.nodes[b].y,c='b')
    plt.scatter(mesh.nodes[b_r].x,mesh.nodes[b_r].y,c='grey')
    plt.scatter(mesh.nodes[b_l].x,mesh.nodes[b_l].y,c='grey')


plt.scatter(mesh.nodes[r].x,mesh.nodes[r].y,c='r')
plt.scatter(mesh.nodes[l].x,mesh.nodes[l].y,c='purple')
plt.scatter(mesh.nodes[sample].x,mesh.nodes[sample].y,c='orange')

dist = np.sqrt((mesh.nodes[sample].x - mesh.nodes[r].x)**2 + (mesh.nodes[sample].y - mesh.nodes[r].y)**2)
span = 2 * dist

xlim = [mesh.nodes[sample].x-span,mesh.nodes[sample].x+span]
ylim = [mesh.nodes[sample].y-span,mesh.nodes[sample].y+span]
ax.set_xlim(xlim)
ax.set_ylim(ylim);

# Stencil based Derivatives

In [ ]:
sample_data = q[:,0]

# set temporary data structures
u_tmp,v_tmp,a_tmp = np.empty(n),np.empty(n),np.empty(n)  
u_tmp,v_tmp,a_tmp = sample_data[:n],sample_data[n:2*n],sample_data[2*n:3*n]

# component to bes examined
ex = u_tmp

# compute spatial derivatives
[dx,dy,ddx,ddy] = mesh.compute_finite_spatial_derivatives(ex,second=True)

# set neighbor nodes
i = sample
u = mesh.nodes[sample].u
b = mesh.nodes[sample].b
r = mesh.nodes[sample].r
l = mesh.nodes[sample].l

# set data from neighbor nodes
if u!=None:
    data_u = ex[u]

if b!=None:
    data_b = ex[b]

data_i = ex[sample]
data_r = ex[r]
data_l = ex[l]

In [ ]:
## GEOMETRIC INFORMATION OF NODES

print_string = "Data Component"
print_padded(print_string)
if u!=None and b!=None:
    print("u:{} \nl:{}\ni:{}\nr:{}\nb:{}".format(data_u,data_l,data_i,data_r,data_b))
elif u!=None:
    print("u:{} \nl:{}\ni:{}\nr:{}".format(data_u,data_l,data_i,data_r))
elif b!=None:
    print("l:{}\ni:{}\nr:{}\nb:{}".format(data_l,data_i,data_r,data_b))

print(" ")

print_string = "Cartesian Coordinates (x,y)"
print_padded(print_string)

coords_i = [mesh.nodes[sample].x,mesh.nodes[sample].y]
coords_r = [mesh.nodes[r].x,mesh.nodes[r].y]
coords_l = [mesh.nodes[l].x,mesh.nodes[l].y]
if u!=None and b!=None:
    coords_u = [mesh.nodes[u].x,mesh.nodes[u].y]
    coords_b = [mesh.nodes[b].x,mesh.nodes[b].y]
    print("u:{} \nl:{}\ni:{}\nr:{}\nb:{}".format(coords_u,coords_l,coords_i,coords_r,coords_b))
elif u!=None:
    coords_u = [mesh.nodes[u].x,mesh.nodes[u].y]
    print("u:{} \nl:{}\ni:{}\nr:{}".format(coords_u,coords_l,coords_i,coords_r))
elif b!=None:
    coords_b = [mesh.nodes[b].x,mesh.nodes[b].y]
    print("l:{}\ni:{}\nr:{}\nb:{}".format(coords_l,coords_i,coords_r,coords_b))

print(" ")

print_string = "Cylinder Coordinates (r,phi)"
print_padded(print_string)

coords_cyl_i = [mesh.nodes[sample].rad,mesh.nodes[sample].theta]
coords_cyl_r = [mesh.nodes[r].rad,mesh.nodes[r].theta]
coords_cyl_l = [mesh.nodes[l].rad,mesh.nodes[l].theta]
if u!=None and b!=None:
    coords_cyl_u = [mesh.nodes[u].rad,mesh.nodes[u].theta]
    coords_cyl_b = [mesh.nodes[b].rad,mesh.nodes[b].theta]
    print("u:{} \nl:{}\ni:{}\nr:{}\nb:{}".format(coords_cyl_u,coords_cyl_l,coords_cyl_i,coords_cyl_r,coords_cyl_b))
elif u!=None:
    coords_cyl_u = [mesh.nodes[u].rad,mesh.nodes[u].theta]
    print("u:{} \nl:{}\ni:{}\nr:{}".format(coords_cyl_u,coords_cyl_l,coords_cyl_i,coords_cyl_r))
elif b!=None:
    coords_cyl_b = [mesh.nodes[b].rad,mesh.nodes[b].theta]
    print("l:{}\ni:{}\nr:{}\nb:{}".format(coords_cyl_l,coords_cyl_i,coords_cyl_r,coords_cyl_b))
print(" ")

### COMPUTING PARTIAL DERIVATIVES 
# TRANSFORM DERIVATIVES
print_string = "Transform Derivatives"
print_padded(print_string)

drdx = coords_i[0]/coords_cyl_i[0]
drdy = coords_i[1]/coords_cyl_i[0]
dphidx = - coords_i[1]/(coords_cyl_i[0]**2)
dphidy = coords_i[0]/(coords_cyl_i[0]**2)
ddrdx = (coords_i[1]**2)/(coords_cyl_i[0]**3)
ddrdy = (coords_i[0]**2)/(coords_cyl_i[0]**3)
ddphidx = (2*coords_i[1]*coords_i[0])/(coords_cyl_i[0]**4)
ddphidy = - (2*coords_i[1]*coords_i[0])/(coords_cyl_i[0]**4)

print("drdx:{}\ndrdy:{}\ndphidx:{}\ndphidy:{}\nddrdx:{}\nddrdy:{}\nddphidx:{}\nddphidy:{}\n".format(drdx,drdy,dphidx,dphidy,ddrdx,ddrdy,ddphidx,ddphidy))

# FINITE DIFFERENCE
print_string = "Finite Difference"
print_padded(print_string)

print("Coordinate: PHI")
if r!=None:
    back_phi = (data_i-data_r)/(coords_cyl_i[1]-coords_cyl_r[1])
    print("Backward:{}".format(back_phi))
if l!=None: 
    forw_phi = (data_l-data_i)/(coords_cyl_l[1]-coords_cyl_i[1])
    print("Forward:{}".format(forw_phi))
if r!=None and l!=None:
    cent_phi = (data_l-data_r)/(coords_cyl_l[1]-coords_cyl_r[1])
    print("Central:{}".format(cent_phi))
    
    cent_sec_phi = (data_u*(coords_cyl_i[1]-coords_cyl_r[1])+data_r*(coords_cyl_l[1]-coords_cyl_i[1])-data_i*(coords_cyl_l[1]-coords_cyl_r[1]))/((coords_cyl_l[1]-coords_cyl_i[1])**2*(coords_cyl_i[1]-coords_cyl_r[1]))
    print("Second Central:{}".format(cent_sec_phi))

print(" ")
print("Coordinate: RADIUS")
if b!=None:
    back_rad = (data_i-data_b)/(coords_cyl_i[0]-coords_cyl_b[0])
    print("Backward:{}".format(back_rad))
if u!=None:
    forw_rad = (data_u-data_i)/(coords_cyl_u[0]-coords_cyl_i[0])
    print("Forward:{}".format(forw_rad))
if u!=None and b!=None:
    cent_rad = (data_u-data_b)/(coords_cyl_u[0]-coords_cyl_b[0])
    print("Central:{}".format(cent_rad))

    cent_sec_rad = (data_u*(coords_cyl_i[0]-coords_cyl_b[0])+data_b*(coords_cyl_u[0]-coords_cyl_i[0])-data_i*(coords_cyl_u[0]-coords_cyl_b[0]))/((coords_cyl_u[0]-coords_cyl_i[0])**2*(coords_cyl_i[0]-coords_cyl_b[0]))
    print("Second Central:{}".format(cent_sec_rad))

print(" ")
print("Coordinate: X")
if b!=None:
    back_dx_fd = back_phi * dphidx + back_rad * drdx
    print("Backward:{}".format(back_dx_fd))
if u!=None:
    forw_dx_fd = forw_phi * dphidx + forw_rad * drdx
    print("Forward:{}".format(forw_dx_fd))
if u!=None and b!=None:
    cent_dx_fd = cent_phi * dphidx + cent_rad * drdx
    print("Central:{}".format(cent_dx_fd))

    ddx_fd = cent_sec_phi * ddphidx + cent_sec_rad * ddrdx + 2*cent_phi*dphidx*cent_rad*drdx
    print("Second:{}".format(ddx_fd))

print(" ")
print("Coordinate: Y")
if b!=None:
    back_dy_fd = back_phi * dphidy + back_rad * drdy
    print("Backward:{}".format(back_dy_fd))
if u!=None:
    forw_dy_fd = forw_phi * dphidy + forw_rad * drdy
    print("Forward:{}".format(forw_dy_fd))
if u!=None and b!=None:
    cent_dy_fd = cent_phi * dphidy + cent_rad * drdy
    print("Central:{}".format(cent_dy_fd))
    
    ddy_fd = cent_sec_phi * ddphidy + cent_sec_rad * ddrdy + 2*cent_phi*dphidy*cent_rad*drdy
    print("Second:{}".format(ddy_fd))

print(" ")
# POLYNOMIAL APPROXIMATION DERIVATIVES
print_string = "POLYNOMIAL APPROXIMATION"
print_padded(print_string)

a_phi = (data_l-data_r)/(coords_cyl_i[1]-coords_cyl_r[1])-(data_i-data_r)/(coords_cyl_l[1]-coords_cyl_r[1])
b_phi = (data_i-data_r)/(coords_cyl_i[1]-coords_cyl_r[1])-(data_l-data_r)+(data_i-data_r)*(coords_cyl_i[1]-coords_cyl_r[1])/(coords_cyl_l[1]-coords_cyl_r[1])
c_phi = data_r
dpol_phi = 2*a_phi*(coords_cyl_i[1]-coords_cyl_r[1]) + b_phi
ddpol_phi = a_phi

print("Coordinate: PHI")
print("Polynomial:{}x²+{}x+{}\nDerivative:{}\nSecond Derivative:{}\n".format(a_phi,b_phi,c_phi,dpol_phi,ddpol_phi))

if u!=None and b!=None:
    print("Coordinate: RADIUS")
    a_rad = (data_u-data_b)/(coords_cyl_i[0]-coords_cyl_b[0])-(data_i-data_b)/(coords_cyl_u[0]-coords_cyl_b[0])
    b_rad = (data_i-data_b)/(coords_cyl_i[0]-coords_cyl_b[0])-(data_l-data_b)+(data_i-data_b)*(coords_cyl_i[0]-coords_cyl_b[0])/(coords_cyl_u[0]-coords_cyl_b[0])
    c_rad = data_r
    dpol_rad = 2*a_rad*(coords_cyl_i[0]-coords_cyl_b[0]) + b_rad
    ddpol_rad = a_rad
    
    print("Polynomial:{}x²+{}x+{}\nDerivative:{}\nSecond Derivative:{}".format(a_rad,b_rad,c_rad,dpol_rad,ddpol_rad))
    
    print(" ")

    print("Coordinate: X")
    dx_pol = dpol_phi * dphidx + dpol_rad * drdx
    ddx_pol = ddpol_phi * ddphidx + ddpol_rad * ddrdx + 2*dpol_phi*dphidx*dpol_rad*drdx
    print("First:{}\nSecond:{}".format(dx_pol,ddx_pol))
    
    print(" ")
    
    print("Coordinate: Y")
    dy_pol = dpol_phi * dphidy + dpol_rad * drdy
    ddy_pol = ddpol_phi * ddphidy + ddpol_rad * ddrdy + 2*dpol_phi*dphidy*dpol_rad*drdy
    print("First:{}\nSecond:{}".format(dy_pol,ddy_pol))

# Flow Fields for different Derivative Methods

In [ ]:
from derivatives import *

In [ ]:
data = q_avg

In [ ]:
# gradient field for finite difference method
finite_differences(mesh,data,fd=True,second=True)
dx_fd = np.empty(n) 
dy_fd = np.empty(n)
laplacian_fd = np.empty(n)
for nod in mesh.nodes:
    dx_fd[nod.index] = nod.dx
    dy_fd[nod.index] = nod.dy
    laplacian_fd[nod.index] = nod.laplacian

In [ ]:
# gradient field for polynomial fitting method
polynomial_derivatives(mesh,data,second=True)
dx_pf = np.empty(n) 
dy_pf = np.empty(n)
laplacian_pf = np.empty(n)
for nod in mesh.nodes:
    dx_pf[nod.index] = nod.dx
    dy_pf[nod.index] = nod.dy
    laplacian_pf[nod.index] = nod.laplacian

In [ ]:
# gradient field for bilinear mapping method
bilinear_derivatives(mesh,data)
dx_bm = np.empty(N) 
dy_bm = np.empty(N)
for cel in mesh.cells:
    dx_bm[cel.index] = cel.dx
    dy_bm[cel.index] = cel.dy

In [ ]:
x = [nod.x for nod in mesh.nodes]
y = [nod.y for nod in mesh.nodes]
x_cell = [cel.center[0] for cel in mesh.cells]
y_cell = [cel.center[1] for cel in mesh.cells]

In [ ]:
plot_cylinder_data(x_cell,y_cell,dx_bm,zoom=True)

In [ ]:
np.where(dx_bm==dx_bm.min())[0][0]